
# 1 - Imports and defining functions

In [2]:
import meshplot as mp
import numpy as np

from pyFM.mesh import TriMesh


def plot_mesh(myMesh, cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist, c=cmap)


def double_plot(myMesh1, myMesh2, cmap1=None, cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)


def visu(vertices):
    min_coord, max_coord = np.min(vertices, axis=0, keepdims=True), np.max(vertices, axis=0, keepdims=True)
    cmap = (vertices - min_coord) / (max_coord - min_coord)
    return cmap


# 2- Loading and processing a mesh

### Basic Mesh methods

A TriMesh class can be created from a path (to a .off or a .obj file) or simply an array of vertices and an optional array of faces.

The mesh can be centered, area-normalized, rotated or translated when loading.


Vertices and faces are stored in the 'vertlist' and 'facelist' attributes. One can also use 'mesh.vertices' and 'mesh.faces' to access them. While these notations can feel non-intuitive they result in clearer functions as it avoids expressions of the form ```mesh.vertices - vertices```.

A TriMesh class possess multiple attributes like edges, per-face area, per-vertex area, per-face normals, per-vertex normals, ...

In [3]:
mesh1 = TriMesh('data/cat1.off', area_normalize=True, center=False)
mesh2 = TriMesh('data/cat-00 small.off', area_normalize=True, center=False)
# pcd2 = TriMesh(np.load('data/cat-00.npy'))


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (14410,) + inhomogeneous part.

In [3]:
mesh1.vertlist.shape

(5000, 3)

In [3]:
plot_mesh(mesh1)
plot_mesh(mesh2)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0211538…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.15…

In [ ]:
# # Attributes are computed on the fly and cached
# edges = mesh1.edges
#
# area = mesh1.area
#
# face_areas = mesh1.face_areas
# vertex_areas = mesh1.vertex_areas
# face_normals = mesh1.normals
#
# # AREA WEIGHTED VERTEX NORMALS
# vertex_normals_a = mesh1.vertex_normals
#
# # UNIFORM WEIGHTED VERTEX NORMALS
# mesh1.set_vertex_normal_weighting('uniform')
# vertex_normals_u = mesh1.vertex_normals

### Geodesics

We propose three versions to compute geodesics :
- Heat method - based on [potpourri3d](https://github.com/nmwsharp/potpourri3d) using robust laplacian (recommended)
- Heat method - pure python implementation from pyFM (not robust but control on the whole code)
- Dijkstra

In [5]:
# Geodesic distance from a given index
# Set robust to False to obtain result from the Python implementation
dists = mesh1.geod_from(1000, robust=True)

In [6]:
S1_geod = mesh1.get_geodesic(verbose=True)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [7]:
S1_geod.shape

(5000, 5000)

### Laplacian and functions

The spectrum of the LBO can be computed easily.

Eigenvalues and eigenvectors are stored in the ```mesh.eigenvalues``` and ```mesh.eigenvectors``` attributes.

Gradient and divergence can be computed using the associated methods. Using the ```mesh.project``` and ```mesh.unproject``` functions allows to switch between seeing a function in the LBO basis or on the complete shape.

The squared $L^2$ norm and $H^1_0$ norm can be computed via the ```mesh.l2_sqnorm``` and ```mesh.h1_sqnorm``` methods.

In [7]:
# By default does not use the intrinsic delaunay Laplacian
mesh1.process(k=3, intrinsic=False, verbose=True); #should replace by spectralnet

In [10]:
# plot the third eigenfunction
plot_mesh(mesh1, mesh1.eigenvectors[:,:])

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-1.359730…

# 3 - Computing the functional map

**Loading data**

In [11]:
# mesh1 = TriMesh(np.load('data/lion-00_1e5.npy'))
# mesh2 = TriMesh(np.load('data/cat-00_1e5.npy'))
mesh1 = TriMesh('data/cat-00.off', area_normalize=True, center=False)
mesh2 = TriMesh('data/lion-00.off')

print(f'Mesh 1 : {mesh1.n_vertices:4d} vertices, {mesh1.n_faces:5d} faces\n'
      f'Mesh 2 : {mesh2.n_vertices:4d} vertices, {mesh2.n_faces:5d} faces')

double_plot(mesh1,mesh2)

Mesh 1 : 7207 vertices, 14410 faces
Mesh 2 : 5000 vertices,  9996 faces


**Computing descriptors**

In [12]:
from pyFM.functional import FunctionalMapping

process_params = {
    'n_ev': (35,35),  # Number of eigenvalues on source and Target
    'landmarks': np.loadtxt('data/landmarks.txt',dtype=int)[:5],  # loading 5 landmarks
    'subsample_step': 4,  # In order not to use too many descriptors
    'descr_type': 'WKS',  # WKS or HKS
}

model = FunctionalMapping(mesh1,mesh2)
model.preprocess(**process_params,verbose=True);


Computing Laplacian spectrum

Computing descriptors
	Normalizing descriptors

	150 out of 600 possible descriptors kept


In [14]:
# model.mesh1.load_eigen_vectors('eigen_vectors_lion_1e5.npy')
# model.mesh2.load_eigen_vectors('eigen_vectors_cat_1e5.npy')
# model.mesh1.load_eigen_vectors('data/emb_cat_off_35.npy')
# model.mesh2.load_eigen_vectors('data/emb_lion_off_35.npy')
# np.load('data/emb_cat_off_35.npy').shape

**Fitting the model**

$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\Ss}{\mathcal{S}}$
$\newcommand{\uargmin}[1]{\underset{#1}{\text{argmin}}\;}$
$\newcommand{\uargmax}[1]{\underset{#1}{\text{argmax}}\;}$
$\def\*#1{\mathbf{#1}}$

In pyFM, we always consider functional maps $\*C:\Ss_1\to\Ss_2$ and pointwise maps $T:\Ss_2\to\Ss_1$ going in opposite directions, with $\*C$ always going from shape 1 to shape 2 !

Optimization problem is
\begin{equation}
\uargmin{\*C\in\RR^{k_2\times k_1}} w_{descr}\|\*C\*A - \*B\|^2 + w_{lap}\|\*C\Delta_1 - \Delta_2\*C\|^2 + w_{\text{d- comm}}\sum_i \|\*C\Gamma_1^i - \Gamma_2^i\*C\|^2 + w_{\text{orient}}\sum_i \|\*C\Lambda_1^i - \Lambda_2^i\*C\|^2
\end{equation}

with $\Gamma_1^i$ and $\Gamma_2^i$ [multipliative operators](http://www.lix.polytechnique.fr/~maks/papers/fundescEG17.pdf) associated to the $i$-th descriptors, $\Lambda_1^i$ and $\Lambda_2^i$ [orientation preserving operators](https://arxiv.org/abs/1806.04455) associated to the $i$-th descriptors

In [13]:
fit_params = {
    'w_descr': 1e-1,
    'w_lap': 1e-2,
    'w_dcomm': 1e-1,
    'w_orient': 0
}

model.fit(**fit_params, verbose=True)

Computing commutativity operators
	Scaling LBO commutativity weight by 8.6e-09

Optimization :
	35 Ev on source - 35 Ev on Target
	Using 150 Descriptors
	Hyperparameters :
		Descriptors preservation :1.0e-01
		Descriptors commutativity :1.0e-01
		Laplacian commutativity :1.0e-02
		Orientation preservation :0.0e+00

	Task : CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH, funcall : 202, nit : 193, warnflag : 0
	Done in 1.95 seconds


**Visualizing the associated point to point map**

In [14]:
p2p_21 = model.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)

# 4 - Refining the Functional Map
```model.FM``` returns the current state of functional map. One can change which one is returned by using ```model.change_FM_type(FM_type)```, as one can see below. 

**ICP**

In [15]:
model.icp_refine(verbose=True)
p2p_21_icp = model.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_icp]
double_plot(mesh1,mesh2,cmap1,cmap2)

  0%|          | 0/10 [00:00<?, ?it/s]

**Zoomout**

In [16]:
model.change_FM_type('classic') # We refine the first computed map, not the icp-refined one
model.zoomout_refine(nit=15, step = 1, verbose=True)
print(model.FM.shape)
p2p_21_zo = model.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_zo]
double_plot(mesh1,mesh2,cmap1,cmap2)

  0%|          | 0/15 [00:00<?, ?it/s]

(50, 50)


# Evaluating Results

In [17]:
import pyFM.eval

In [18]:
# Compute geodesic distance matrix on the cat mesh
A_geod = mesh1.get_geodesic(verbose=True)

  0%|          | 0/7207 [00:00<?, ?it/s]

In [19]:
# Load an approximate ground truth map
gt_p2p = np.loadtxt('data/lion2cat',dtype=int)

acc_base = pyFM.eval.accuracy(p2p_21, gt_p2p, A_geod, sqrt_area=mesh1.sqrtarea)

acc_icp = pyFM.eval.accuracy(p2p_21_icp, gt_p2p, A_geod, sqrt_area=np.sqrt(mesh1.area))

acc_zo = pyFM.eval.accuracy(p2p_21_zo, gt_p2p, A_geod, sqrt_area=np.sqrt(mesh1.area))

print(f'Accuracy results\n'
      f'\tBasic FM : {1e3*acc_base:.2f}\n'
      f'\tICP refined : {1e3*acc_icp:.2f}\n'
      f'\tZoomOut refined : {1e3*acc_zo:.2f}\n')

Accuracy results
	Basic FM : 99.19
	ICP refined : 71.14
	ZoomOut refined : 62.71



In [2]:
from pyFM.functional import FunctionalMapping
import meshplot as mp
import numpy as np

from pyFM.mesh import TriMesh


def plot_mesh(myMesh, cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist, c=cmap)


def double_plot(myMesh1, myMesh2, cmap1=None, cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)


def visu(vertices):
    min_coord, max_coord = np.min(vertices, axis=0, keepdims=True), np.max(vertices, axis=0, keepdims=True)
    cmap = (vertices - min_coord) / (max_coord - min_coord)
    return cmap

mesh1 = TriMesh('data/cat-00.off', area_normalize=True, center=False)
mesh2 = TriMesh('data/lion-00.off')
# mesh1 = TriMesh(np.load('data/lion-00_1e5.npy'))
# mesh2 = TriMesh(np.load('data/cat-00_1e5.npy'))
process_params = {
    'n_ev': (20,20),  # Number of eigenvalues on source and Target
    'landmarks': np.loadtxt('data/landmarks.txt',dtype=int)[:5],  # loading 5 landmarks
    'subsample_step': 4,  # In order not to use too many descriptors
    'descr_type': 'WKS',  # WKS or HKS
    'k_process' : 20 # Number of eigenvalues to compute
}
model = FunctionalMapping(mesh1,mesh2)
model.preprocess(**process_params,verbose=True, is_point_cloud=False, intrinsic=False)
fit_params = {
    'w_descr': 1e-1,
    'w_lap': 1e-2,
    'w_dcomm': 1e-1,
    'w_orient': 0
}

model.fit(**fit_params, verbose=True)
p2p_21 = model.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist)
cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)

# model.icp_refine(verbose=True)
# p2p_21_icp = model.get_p2p()
# cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_icp]
# double_plot(mesh1,mesh2,cmap1,cmap2)


Computing Laplacian spectrum
Functional map eigenvalues:
[1.47287734e-04 6.27076912e+00 1.19215260e+01 1.85784302e+01
 2.34039154e+01 2.40148258e+01 3.09045715e+01 4.87182846e+01
 7.56455383e+01 7.59310074e+01 7.67824936e+01 9.85075226e+01
 1.04412010e+02 1.09218819e+02 1.26275345e+02 1.44637726e+02
 1.53799393e+02 1.56067307e+02 1.75834045e+02 1.81509674e+02]
Functional map eigenvalues sum: 1642.4332
Training SpectralNet:


  1%|          | 1/100 [00:00<01:16,  1.29it/s]

Train Loss epoch 0: 4419.28662109375, LR: 0.01


  2%|▏         | 2/100 [00:02<02:07,  1.30s/it]

Train Loss epoch 1: 4154.109375, LR: 0.01


...

 99%|█████████▉| 99/100 [00:59<00:00,  1.62it/s]

Train Loss epoch 98: 1635.4388427734375, LR: 0.005


100%|██████████| 100/100 [01:00<00:00,  1.65it/s]

Train Loss epoch 99: 1633.3148193359375, LR: 0.005


Spectral net eigenvalues sum: 496.37576
grassmann: 2.050153338561036
Functional map eigenvalues:
[-3.7238002e-05  1.0874743e+01  1.8151787e+01  2.9111294e+01
  3.0641777e+01  3.1309780e+01  4.7858860e+01  8.7912872e+01
  1.4054167e+02  1.4803436e+02  1.4944655e+02  1.7319580e+02
  1.7674756e+02  2.0749742e+02  2.4568953e+02  2.7712427e+02
  3.0069211e+02  3.0265152e+02  3.3201282e+02  3.7768405e+02]
Functional map eigenvalues sum: 3087.1787
Spectral net eigenvalues sum: 496.37576
grassmann: 4.221486816688682

Computing descriptors
	Normalizing descriptors

	150 out of 600 possible descriptors kept
Computing commutativity operators
	Scaling LBO commutativity weight by 2.8e-06

Optimization :
	20 Ev on source - 20 Ev on Target
	Using 150 Descriptors
	Hyperparameters :
		Descriptors preservation :1.0e-01
		Descriptors commutativity :1.0e-01
		Laplacian commutativity :1.0e-02
		Orientation preservation :0.0e+00

	Task : CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL, funcall : 31, nit : 